In [25]:
import time
import numpy as np
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait


def document_initialised(driver):
    return driver.execute_script("return initialised")

driver = webdriver.Chrome(ChromeDriverManager().install())

sainsburys_beer_path = 'https://www.sainsburys.co.uk/shop/gb/groceries/drinks/all-beers'

driver.get(sainsburys_beer_path)

time.sleep(1)

WebDriverWait(driver, timeout=10).until(lambda d: d.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))

driver.find_element(By.XPATH, '//*[@id="onetrust-accept-btn-handler"]').click()

elems = driver.find_elements(By.XPATH,"//a[@href]")

# get links for all beers in the page so we can start visiting beer level information
links = []
for elem in elems:
    pattern = 'https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers'
    try:
        attr = elem.get_attribute("href")
        if pattern in attr:
            links.append(attr)
    except:
        pass

def get_abv(abv):
    for t in abv:
        if 'Alcohol' in t.text:
            return t
data = []

# Get %Alcohol and price per liter values 
for l in np.unique(links):
    try:
        driver.get(l) 
        time.sleep(3) 
        x_path_alcohol_volume = '//*[@id="mainPart"]/div[4]/div/p'

        ppl = driver.find_element(By.CLASS_NAME,'pd__cost__per-unit')
        abv_0 = driver.find_elements(By.CLASS_NAME,'nameTextItems')
        abv_1 = get_abv(abv_0)
        entry = [l, ppl.text, abv_1.text]
        data.append(entry)
        print(l, ppl.text, abv_1.text)
    except:
        pass




/var/folders/p6/325r0tsd2cd1jddpt9nd70hm0000gn/T/ipykernel_48388/1563364347.py:12: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/badger-fursty-ferret-ale-500ml £3.30 / ltr Alcohol by volume: 4.4%
Serving suggestion: Dark crystal malt delivers a full bodied biscuity taste, rounded off with a subtle floral and lemon hop aroma
Alcoholic content: 2.2 units
https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/birra-moretti-12x330ml £3.79 / ltr Alcohol by volume: 4.6%
Alcoholic content: 1.5 units
https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/birra-moretti-lager-4x330ml £3.98 / ltr Alcohol by volume: 4.6%
Alcoholic content: 1.5 units
https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/birra-moretti-lager-660ml £2.88 / ltr Alcohol by volume: 4.6%
https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/birra-moretti-lager-beer-bottle-18x330ml £2.86 / ltr Alcohol by volume: 4.6%
https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/bishops-finger-a

In [85]:
# Cleanup

import pandas as pd
import re
df = pd.DataFrame(data=data, columns=['href','ppl','abv_raw'])

abv_raw_list  = df['abv_raw'].to_list()
abv_list = []

for i in abv_raw_list:
    m = re.findall('([0-9]*\.?[0-9]*)\s*%',i)
    if len(m) > 0:
        abv_list.append(list(m)[0])
    else:
        abv_list.append('NaN')


In [108]:
# Conver ppl values into workable numbers

df['ppl_int']= pd.to_numeric(df['ppl'].apply(lambda x: x[1:5])) 

df['price_per_unit'] = df['ppl_int']/pd.to_numeric(df['abv'])

# Extract beer names

df['beer_name'] = df['href'].apply(lambda x: x.replace('https://www.sainsburys.co.uk/shop/gb/groceries/product/details/all-beers/',''))

df.sort_values(by='price_per_unit')


,href,ppl,abv_raw,abv,ppl_int,price_per_unit,beer_name
28,https://www.sainsburys.co.uk/shop/gb/groceries...,£1.51 / ltr,Alcohol by volume: 5.0%,5.0,1.51,0.302000,kronenbourg-1664-premium-beer-15-x-440ml-79613...
10,https://www.sainsburys.co.uk/shop/gb/groceries...,£1.39 / ltr,Alcohol by volume: 4%,4,1.39,0.347500,carling-lager-18x440ml
48,https://www.sainsburys.co.uk/shop/gb/groceries...,£1.64 / ltr,Alcohol by volume: 4.5%,4.5,1.64,0.364444,strongbow-original-cider-18x440ml
47,https://www.sainsburys.co.uk/shop/gb/groceries...,£1.77 / ltr,Alcohol by volume: 4.8%\nServing suggestion: A...,4.8,1.77,0.368750,stella-artois-premium-lager-beer-cans-18x440ml
18,https://www.sainsburys.co.uk/shop/gb/groceries...,£1.51 / ltr,Alcohol by volume: 4.0%,4.0,1.51,0.377500,fosters-lager-18x440ml
8,https://www.sainsburys.co.uk/shop/gb/groceries...,£1.77 / ltr,Alcohol by volume: 4.5%,4.5,1.77,0.393333,budweiser-18x440ml
50,https://www.sainsburys.co.uk/shop/gb/groceries...,£1.82 / ltr,Alcohol by volume: 4.5%\nServing suggestion: T...,4.5,1.82,0.404444,thatchers-haze-cloudy-somerset-cider-10x440ml
9,https://www.sainsburys.co.uk/shop/gb/groceries...,£1.83 / ltr,Alcohol by volume: 4.5%\nAlcoholic content: 1....,4.5,1.83,0.406667,budweiser-lager-beer-bottles-20x300ml
23,https://www.sainsburys.co.uk/shop/gb/groceries...,£2.27 / ltr,Alcohol by volume: 5%,5,2.27,0.454000,heineken-12x330ml
25,https://www.sainsburys.co.uk/shop/gb/groceries...,£3.75 / ltr,Alcohol by volume: 8.2%\nServing suggestion: S...,8.2,3.75,0.457317,henry-westons-vintage-cider-4x500ml


In [119]:
df.to_csv('beers.csv')